# CS 634 Midterm Project              By Chike Egbuchulam(coe5)

## Setup 

First you want to import the libraries essential to run the the FP and Apriori Algorithms

In [1]:
import os
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori,fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
import time
import itertools
from bfassociationrules import assocRules

### Set  the initital variables as well as the directory

In [2]:
dbfile = ""
stores = ["Alice's Bakery", "Bob's Breakfast", "Charlie's Cafe", "Desayuno de David", "Eddie's Eatery"]
supportValue, confidenceValue = 0, 0

base_dir = os.getcwd()
tables_dir = os.path.join(base_dir, "tables")

print("Base directory:", base_dir)
print("Tables directory:", tables_dir)

Base directory: C:\Users\egbuc\cs634\Egbuchulam_Chike_midtermproject
Tables directory: C:\Users\egbuc\cs634\Egbuchulam_Chike_midtermproject\tables


### Now let's get the Minimum Support and Minimimum Confidence values

In [3]:
while True:
    sV = input("Please enter the minimum support % you want (0-100): ")
    try:
        supportValue = float(sV) / 100
    except ValueError:
        print("Please input a numeric value")
        continue
    if 0.0 <= supportValue <= 1.00:
        break
    else:
        print("Invalid selection")

while True:
    cV = input("Please enter the minimum confidence % you want (0-100): ")
    try:
        confidenceValue = float(cV) / 100
    except ValueError:
        print("Please input a numeric value")
        continue
    if 0.0 <= confidenceValue <= 1.0:
        break
    else:
        print("Invalid selection")
print("Support {} %".format(supportValue * 100))
print("Confidence {} %".format(confidenceValue * 100))

Please enter the minimum support % you want (0-100):  10
Please enter the minimum confidence % you want (0-100):  20


Support 10.0 %
Confidence 20.0 %


### Now we'll run theses values for each database with our algorithm while also running the aprioiri and FP growth algorithms for comparison

## Alice's Bakery

 Here is the transaction table for Alice's bakery

### Here is the transaction table for Alice's bakery

In [4]:
dbfile = os.path.join(tables_dir, "alicebakery001.csv")
transactions = pd.read_csv(dbfile, usecols=["ItemsPurchased"])
transactions_set = transactions["ItemsPurchased"].apply(lambda x: set(item.strip() for item in x.split(","))).tolist()
unique_items = set().union(*transactions_set)
print(transactions_set)

[{'Bread'}, {'Scandinavian'}, {'Jam', 'Hot chocolate', 'Cookies'}, {'Muffin'}, {'Bread', 'Coffee', 'Pastry'}, {'Medialuna', 'Pastry', 'Muffin'}, {'Medialuna', 'Pastry', 'Coffee', 'Tea'}, {'Bread', 'Pastry'}, {'Muffin', 'Bread'}, {'Medialuna', 'Scandinavian'}, {'Medialuna', 'Bread'}, {'Tea', 'Pastry', 'Jam', 'Coffee'}, {'Bread', 'Coffee'}, {'Medialuna', 'Bread', 'Pastry'}, {'Mineral water', 'Scandinavian'}, {'Medialuna', 'Bread', 'Coffee'}, {'Hot chocolate'}, {'Farm House'}, {'Bread', 'Farm House'}, {'Medialuna', 'Bread'}, {'Medialuna', 'Bread', 'Coffee'}, {'Jam'}, {'Muffin', 'Scandinavian'}, {'Bread'}, {'Scandinavian'}, {'Fudge'}, {'Scandinavian'}, {'Bread', 'Coffee'}, {'Bread', 'Jam'}, {'Bread'}, {'Muffin', 'Scandinavian'}, {'Coffee'}, {'Muffin', 'Coffee'}, {'Muffin', 'Scandinavian'}, {'Tea', 'Bread'}, {'Bread', 'Coffee'}, {'Tea', 'Bread'}, {'Scandinavian'}, {'Muffin', 'Juice', 'Coffee'}, {'Scandinavian'}]


### Now lets run the algorithm with the values

In [5]:
bStart = time.perf_counter()
[fSets,rules] = assocRules(unique_items,transactions_set,supportValue,confidenceValue)
bEnd = time.perf_counter()


### Here are the frequent sets:

In [6]:
print(fSets)

[{'Medialuna', 'Bread'}, {'Bread', 'Coffee'}]


### Here are the rules:

In [7]:
if(rules != [] and supportValue!=0):
    i = 1
    for (a, b, c) in rules:
        print(f"Rule {i}: {a} ===> {b} \t Confidence :{c}")
        print("\n")
        i += 1
else:
    if supportValue == 0:
            print(
                "At 0% support I have to look at possible menu pairing as a rule even if it happens only once. "
                "So in other words, you're trying to waste my time so here is the whole list of transactions."
            )
            # print(transactions)
    
    if supportValue != 0 and len(rules) == 0:
        print("There are no rules that satisfy these minimum values")

Rule 1: Medialuna ===> Bread 	 Confidence :1.0


Rule 2: Bread ===> Medialuna 	 Confidence :0.5


Rule 3: Bread ===> Coffee 	 Confidence :0.5


Rule 4: Coffee ===> Bread 	 Confidence :1.0




### Here's how it compares to other algorithms

In [8]:
tList = transactions["ItemsPurchased"].apply(lambda x: [item.strip() for item in x.split(",")]).tolist()
te = TransactionEncoder()
te_array = te.fit(tList).transform(tList)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

aStart = time.perf_counter()
frequent_items = apriori(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
aEnd = time.perf_counter()
print(f"Apriori check:\n {frequent_items}")
fStart= time.perf_counter()
frequent_items = fpgrowth(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
fEnd = time.perf_counter() 

print(f"Our Association Algorithm: {bEnd - bStart:.4f} seconds")
print(f"Apriori:{aEnd - aStart:.4f} seconds")
print(f"FP-Growth:{fEnd - fStart: .4f} seconds")

Apriori check:
    support            itemsets
0    0.450             (Bread)
1    0.275            (Coffee)
2    0.100               (Jam)
3    0.200         (Medialuna)
4    0.200            (Muffin)
5    0.150            (Pastry)
6    0.250      (Scandinavian)
7    0.100               (Tea)
8    0.150     (Bread, Coffee)
9    0.125  (Medialuna, Bread)
Our Association Algorithm: 0.0033 seconds
Apriori:0.0204 seconds
FP-Growth: 0.0161 seconds


### Now for the rest of the tables

## Bob's Breakfast

In [9]:
dbfile = os.path.join(tables_dir, "bobsbreakfast001.csv")
transactions = pd.read_csv(dbfile, usecols=["ItemsPurchased"])
transactions_set = transactions["ItemsPurchased"].apply(lambda x: set(item.strip() for item in x.split(","))).tolist()
unique_items = set().union(*transactions_set)
print(dbfile)

bStart = time.perf_counter()
[fSets,rules] = assocRules(unique_items,transactions_set,supportValue,confidenceValue)
bEnd = time.perf_counter()


print(f"Frequent Sets: {fSets}")
print('\n')

if(rules != [] and supportValue!=0):
    i = 1
    for (a, b, c) in rules:
        print(f"Rule {i}: {a} ===> {b} \t Confidence :{c}")
        print("\n")
        i += 1
else:
    if supportValue == 0:
        print(
            "At 0% support I have to look at possible menu pairing as a rule even if it happens only once. "
            "So in other words, you're trying to waste my time so here is the whole list of transactions."
        )
        # print(transactions)
    
    if supportValue != 0 and len(rules) == 0:
        print("There are no rules that satisfy these minimum values")

C:\Users\egbuc\cs634\Egbuchulam_Chike_midtermproject\tables\bobsbreakfast001.csv
Frequent Sets: [{'Tea', 'Coffee'}]


Rule 1: Coffee ===> Tea 	 Confidence :1.0


Rule 2: Tea ===> Coffee 	 Confidence :1.0




#### Comparisons

In [10]:
tList = transactions["ItemsPurchased"].apply(lambda x: [item.strip() for item in x.split(",")]).tolist()
te = TransactionEncoder()
te_array = te.fit(tList).transform(tList)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

aStart = time.perf_counter()
frequent_items = apriori(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
aEnd = time.perf_counter()
print(f"Apriori check:\n {frequent_items}")
fStart= time.perf_counter()
frequent_items = fpgrowth(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
fEnd = time.perf_counter() 

print(f"Our Association Algorithm: {bEnd - bStart:.4f} seconds")
print(f"Apriori:{aEnd - aStart:.4f} seconds")
print(f"FP-Growth:{fEnd - fStart: .4f} seconds")

Apriori check:
    support             itemsets
0    0.275              (Bread)
1    0.475             (Coffee)
2    0.100              (Fudge)
3    0.100  (Hearty & Seasonal)
4    0.100             (Muffin)
5    0.125       (Scandinavian)
6    0.200                (Tea)
7    0.100        (Tea, Coffee)
Our Association Algorithm: 0.0037 seconds
Apriori:0.0156 seconds
FP-Growth: 0.0136 seconds


## Charlie's Cafe

In [11]:
dbfile = os.path.join(tables_dir, "charliescafe001.csv")
transactions = pd.read_csv(dbfile, usecols=["ItemsPurchased"])
transactions_set = transactions["ItemsPurchased"].apply(lambda x: set(item.strip() for item in x.split(","))).tolist()
unique_items = set().union(*transactions_set)
print(dbfile)

bStart = time.perf_counter()
[fSets,rules] = assocRules(unique_items,transactions_set,supportValue,confidenceValue)
bEnd = time.perf_counter()


print(f"Frequent Sets: {fSets}")
print('\n')

if(rules != [] and supportValue!=0):
    i = 1
    for (a, b, c) in rules:
        print(f"Rule {i}: {a} ===> {b} \t Confidence :{c}")
        print("\n")
        i += 1
else:
    if supportValue == 0:
        print(
            "At 0% support I have to look at possible menu pairing as a rule even if it happens only once. "
            "So in other words, you're trying to waste my time so here is the whole list of transactions."
        )
        # print(transactions)

    if supportValue != 0 and len(rules) == 0:
        print("There are no rules that satisfy these minimum values")

C:\Users\egbuc\cs634\Egbuchulam_Chike_midtermproject\tables\charliescafe001.csv
Frequent Sets: [{'Coffee', 'Pastry'}, {'Bread', 'Coffee'}]


Rule 1: Pastry ===> Coffee 	 Confidence :1.0


Rule 2: Bread ===> Coffee 	 Confidence :1.0


Rule 3: Coffee ===> Pastry 	 Confidence :0.5


Rule 4: Coffee ===> Bread 	 Confidence :0.5




#### Comparisons

In [12]:
tList = transactions["ItemsPurchased"].apply(lambda x: [item.strip() for item in x.split(",")]).tolist()
te = TransactionEncoder()
te_array = te.fit(tList).transform(tList)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

aStart = time.perf_counter()
frequent_items = apriori(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
aEnd = time.perf_counter()
print(f"Apriori check:\n {frequent_items}")
fStart= time.perf_counter()
frequent_items = fpgrowth(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
fEnd = time.perf_counter() 

print(f"Our Association Algorithm: {bEnd - bStart:.4f} seconds")
print(f"Apriori:{aEnd - aStart:.4f} seconds")
print(f"FP-Growth:{fEnd - fStart: .4f} seconds")

Apriori check:
    support          itemsets
0    0.300           (Bread)
1    0.525          (Coffee)
2    0.150      (Farm House)
3    0.100          (Muffin)
4    0.250          (Pastry)
5    0.125             (Tea)
6    0.100   (Bread, Coffee)
7    0.150  (Pastry, Coffee)
Our Association Algorithm: 0.0023 seconds
Apriori:0.0167 seconds
FP-Growth: 0.0144 seconds


## Desayuno de David

In [13]:
dbfile = os.path.join(tables_dir, "desayunodedavid001.csv")
transactions = pd.read_csv(dbfile, usecols=["ItemsPurchased"])
transactions_set = transactions["ItemsPurchased"].apply(lambda x: set(item.strip() for item in x.split(","))).tolist()
unique_items = set().union(*transactions_set)
print(dbfile)

bStart = time.perf_counter()
[fSets,rules] = assocRules(unique_items,transactions_set,supportValue,confidenceValue)
bEnd = time.perf_counter()


print(f"Frequent Sets: {fSets}")
print('\n')

if(rules != [] and supportValue!=0):
    i = 1
    for (a, b, c) in rules:
        print(f"Rule {i}: {a} ===> {b} \t Confidence :{c}")
        print("\n")
        i += 1
else:
    if supportValue == 0:
        print(
            "At 0% support I have to look at possible menu pairing as a rule even if it happens only once. "
            "So in other words, you're trying to waste my time so here is the whole list of transactions."
        )
        # print(transactions)

    if supportValue != 0 and len(rules) == 0:
        print("There are no rules that satisfy these minimum values")

C:\Users\egbuc\cs634\Egbuchulam_Chike_midtermproject\tables\desayunodedavid001.csv
Frequent Sets: [{'Muffin', 'Coffee'}]


Rule 1: Muffin ===> Coffee 	 Confidence :1.0


Rule 2: Coffee ===> Muffin 	 Confidence :1.0




#### Comparisons

In [14]:
tList = transactions["ItemsPurchased"].apply(lambda x: [item.strip() for item in x.split(",")]).tolist()
te = TransactionEncoder()
te_array = te.fit(tList).transform(tList)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

aStart = time.perf_counter()
frequent_items = apriori(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
aEnd = time.perf_counter()
print("Apriori check:{frequent_items}")
fStart= time.perf_counter()
frequent_items = fpgrowth(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
fEnd = time.perf_counter() 

print(f"Our Association Algorithm: {bEnd - bStart:.4f} seconds")
print(f"Apriori:{aEnd - aStart:.4f} seconds")
print(f"FP-Growth:{fEnd - fStart: .4f} seconds")

Apriori check:{frequent_items}
Our Association Algorithm: 0.0036 seconds
Apriori:0.0167 seconds
FP-Growth: 0.0129 seconds


## Eddie's Eats

In [15]:
dbfile = os.path.join(tables_dir, "eddieseats001.csv")
transactions = pd.read_csv(dbfile, usecols=["ItemsPurchased"])
transactions_set = transactions["ItemsPurchased"].apply(lambda x: set(item.strip() for item in x.split(","))).tolist()
unique_items = set().union(*transactions_set)
print(dbfile)

bStart = time.perf_counter()
[fSets,rules] = assocRules(unique_items,transactions_set,supportValue,confidenceValue)
bEnd = time.perf_counter()


print(f"Frequent Sets: {fSets}")
print('\n')

if(rules != [] and supportValue!=0):
    i = 1
    for (a, b, c) in rules:
        print(f"Rule {i}: {a} ===> {b} \t Confidence :{c}")
        print("\n")
        i += 1
else:
    if supportValue == 0:
        print(
            "At 0% support I have to look at possible menu pairing as a rule even if it happens only once. "
            "So in other words, you're trying to waste my time so here is the whole list of transactions."
        )
        # print(transactions)

    if supportValue != 0 and len(rules) == 0:
        print("There are no rules that satisfy these minimum values")

C:\Users\egbuc\cs634\Egbuchulam_Chike_midtermproject\tables\eddieseats001.csv
Frequent Sets: [{'Coffee', 'Pastry'}]


Rule 1: Pastry ===> Coffee 	 Confidence :1.0


Rule 2: Coffee ===> Pastry 	 Confidence :1.0




#### Comparisons

In [16]:
tList = transactions["ItemsPurchased"].apply(lambda x: [item.strip() for item in x.split(",")]).tolist()
te = TransactionEncoder()
te_array = te.fit(tList).transform(tList)
df_encoded = pd.DataFrame(te_array, columns=te.columns_)

aStart = time.perf_counter()
frequent_items = apriori(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
aEnd = time.perf_counter()
print(f"Apriori check:\n {frequent_items}")
fStart= time.perf_counter()
frequent_items = fpgrowth(df_encoded, min_support=supportValue, use_colnames=True)
rules = association_rules(frequent_items, metric="confidence", min_threshold=confidenceValue)
fEnd = time.perf_counter() 

print(f"Our Association Algorithm: {bEnd - bStart:.4f} seconds")
print(f"Apriori:{aEnd - aStart:.4f} seconds")
print(f"FP-Growth:{fEnd - fStart: .4f} seconds")

Apriori check:
    support          itemsets
0    0.275           (Bread)
1    0.500          (Coffee)
2    0.175          (Pastry)
3    0.250             (Tea)
4    0.100  (Pastry, Coffee)
Our Association Algorithm: 0.0039 seconds
Apriori:0.0158 seconds
FP-Growth: 0.0084 seconds
